## Merge database tables into a flat dataframe

In [1]:
import sqlite3
import pandas as pd

pd.set_option("display.max_colwidth", 60)
pd.set_option("display.max_rows", 100)

In [2]:
RELEASE_DB_NAME = "release_db.sqlite"
conn = sqlite3.connect(RELEASE_DB_NAME)
conn

### Read in the tables

In [3]:
# we'll merge everything into this dataframe
df = pd.read_sql_query("SELECT * FROM policy_snapshots", conn)

sites_df = pd.read_sql_query("SELECT * FROM sites", conn)
policy_texts_df = pd.read_sql_query("SELECT * FROM policy_texts", conn)
alexa_ranks_df = pd.read_sql_query("SELECT * FROM alexa_ranks", conn)

In [4]:
# Left join with policy text table
df = pd.merge(df, policy_texts_df, how="left", left_on="policy_text_id", right_on="id")

# Left join with sites table
df = pd.merge(df, sites_df, how="left", left_on="site_id", right_on="id")

# Left join with alexa ranks table
df = pd.merge(df, alexa_ranks_df, how="left", on=['site_id', 'year', 'phase'])

In [5]:
df_simplified = df[['id_x', 'analysis_subcorpus', 'domain', 'rank', 'year', 'phase','categories', 'flesch_kincaid','flesch_ease', 'smog', 'length', 'cross_domain_homepage_redir']]
df_simplified = df_simplified.loc[df_simplified['analysis_subcorpus'] == 1]

df_simplified['Year Phase'] = df['year'].astype(str) + df['phase']
df_simplified['Organisation'] = df_simplified['domain'].str.split('.').str[0]
df_simplified['rank (bin)'] = pd.cut(df_simplified['rank'], [0, 1000, 10000, 100000, 1000000])

df_simplified.head(20)

,id_x,analysis_subcorpus,domain,rank,year,phase,categories,flesch_kincaid,flesch_ease,smog,length,cross_domain_homepage_redir,Year Phase,Organisation,rank (bin)
0,1,1,docusign.com,NaN,2003,B,business;informationtech,14.938384,difficult,18.351184,14137,0,2003B,docusign,NaN
1,2,1,docusign.com,NaN,2004,B,business;informationtech,13.762202,difficult,18.171460,14366,0,2004B,docusign,NaN
2,3,1,docusign.com,NaN,2006,B,business;informationtech,13.640407,difficult,18.062587,14363,0,2006B,docusign,NaN
3,4,1,docusign.com,NaN,2006,A,business;informationtech,13.640407,difficult,18.062587,14363,0,2006A,docusign,NaN
4,5,1,docusign.com,NaN,2004,A,business;informationtech,13.762202,difficult,18.171460,14366,0,2004A,docusign,NaN
5,6,1,docusign.com,NaN,2005,A,business;informationtech,13.651666,difficult,18.062587,14394,0,2005A,docusign,NaN
6,7,1,docusign.com,NaN,2008,A,business;informationtech,13.660300,difficult,18.098966,14396,0,2008A,docusign,NaN
7,8,1,docusign.com,NaN,2007,A,business;informationtech,13.640407,difficult,18.062587,14363,0,2007A,docusign,NaN
8,9,1,docusign.com,NaN,2007,B,business;informationtech,13.655114,difficult,18.062587,14391,0,2007B,docusign,NaN
9,10,1,docusign.com,NaN,2008,B,business;informationtech,13.660300,difficult,18.098966,14396,0,2008B,docusign,NaN


In [6]:
df_simplified.to_csv('simplified_policy_data.csv')